# Viscous liquid transfer report

The objective of this Jupyter is to guide the user through the process of obtaining the liquid handling report for OT2 pipettes for a specific viscous liquids. The process is based on a gravimetric method where the volume transferred with the OT2 pipettes is compared with the mass transferred during pipetting, allowing to compute a relative transfer error for each set of liquid handling parameters. 

In [ ]:
import os
import json
import opentrons.execute
import pandas as pd
import numpy as np
import time

In [1]:
def transfer_viscous_liquid(liquid, volume, source, destination, pipette, distance, new_tip=True):
    """Function to transfer viscous liquids using a OT2 robot (v2).
    Args:
        liquid: key in liquids_dict for liquid handling parameter definitions.
        volume: target volume to be transferred.
        source/destination: point object of OT2 that define place of aspiration and dispense respectively.
        pipette: OT2 InstrumentContext object.
        distance: height of liquid in source vial in mm
        new_tip specifies: True if new tip is required for the transfer.
    """
    if new_tip == True: pipette.pick_up_tip()
    if 'P1' in str(pipette):
        pipette_name = 'p1000'
    elif 'P3'in str(pipette):
        pipette_name = 'p300'
    if pipette.has_tip == False:
        raise Exception("Can't aspirate liquid with no tip")
    pipette.aspirate(volume, source.bottom(distance), rate = liquids_dict[liquid][pipette_name]['aspiration_rate']/pipette.flow_rate.aspirate)
    time.sleep(liquids_dict[liquid][pipette_name]['delay_aspirate'])
    if liquids_dict[liquid][pipette_name]['touch_tip_aspirate'] == True:
        pipette.touch_tip()
    pipette.dispense(volume, destination.top(-5), rate = liquids_dict[liquid][pipette_name]['dispense_rate']/pipette.flow_rate.dispense)
    time.sleep(liquids_dict[liquid][pipette_name]['delay_dispense'])
    if liquids_dict[liquid][pipette_name]['blow_out_rate'] > 0:
        pipette.flow_rate.blow_out = liquids_dict[liquid][pipette_name]['blow_out_rate']
        pipette.blow_out()
        pipette.flow_rate.blow_out = pipette.flow_rate.aspirate 
        time.sleep(liquids_dict[liquid][pipette_name]['delay_blow_out'])
    if liquids_dict[liquid][pipette_name]['touch_tip_dispense'] == True:
        pipette.touch_tip()   
    if new_tip == True:
        pipette.drop_tip()

In [ ]:
#Initialization of API and deck setup
protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
tiprack_1000 = protocol.load_labware('opentrons_96_tiprack_1000ul', 11)
tiprack_300=  protocol.load_labware('opentrons_96_tiprack_300ul', 8)
pipettes = {'p1000' : protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tiprack_1000]), 'p300' : protocol.load_instrument('p300_multi_gen2', 'right', tip_racks=[tiprack_300])}
source = protocol.load_labware('amdm_12_wellplate_30000ul',6) 

In [ ]:
#Stablish starting pipette tips locations
pipettes['p1000'].starting_tip = tiprack_1000.well('B7')
pipettes['p300'].starting_tip = tiprack_300.well('B9')

# Gravimetric calibration of viscous liquid transfer

The following cells contain the code required to implement the gravimetric analysis of volume transfer of a specific viscous liquid. User only needs to update dictionary values for liquid handling parameters, input the target volume, density of the liquid and mass of vials before and after a dispense.

In [ ]:
#Stablish liquid and initial height of liquid on the source vial
liquid = 'Viscosity_std_1275'
density = 0.8736
liquid_level = 53

In [ ]:
#Create DataFrame to record measurements
df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out_rate', 'delay_aspirate',  'delay_dispense', 'delay_blow_out','touch_tip_aspirate', 'touch_tip_dispense', 'density','mi', 'mf', 'm', '%error', 'Comment'])
df = df.astype({'liquid':str,'pipette':str,"touch_tip_aspirate":bool,"touch_tip_dispense":bool,'Comment':str})

Update liquid handling parameters 

In [ ]:
liquids_dict = {
    liquid: {
        "p1000": {
            "aspiration_rate": 4.489998,
            "dispense_rate": 10.881212,
            "blow_out_rate" : 0, 
            "delay_aspirate" : 5, 
            "delay_dispense" : 5, 
            "delay_blow_out" : 0,
            "touch_tip_aspirate": True, 
            "touch_tip_dispense" : False,
        }  
    }
}

In [ ]:
pipette = 'p1000'
volume = 1000 #FILLIN
if pipettes[pipette].has_tip == False:
    pipettes[pipette].pick_up_tip()
transfer_viscous_liquid(liquid, volume, source.wells_by_name()['A1'], source.wells_by_name()['A1'], pipettes[pipette], liquid_level-15, new_tip=False)

pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
protocol.delay(seconds=10)
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())

Transfer viscous liquids, input pipette name (pipette), desired volume (volume) to be dispensed in ul, liquid dictionary key string (liquid), density (density) and initial vial mass (mi). The code will register the liquid handling parameters used into the DataFrame.

In [ ]:
pipette = 'p1000'
volume = 300 #FILLIN
mi = mf #FILLIN

transfer_viscous_liquid(liquid, volume, source.wells_by_name()['A1'], source.wells_by_name()['A4'], pipettes[pipette], liquid_level-15, new_tip=False)
pipettes[pipette].move_to(source.wells_by_name()['A1'].top())
df = df.append(liquids_dict[liquid][pipette], ignore_index = True)

for i in range(2):
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=5)
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=5)
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

    pipettes['p1000'].home_plunger()
    protocol.delay(seconds=5)
    pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
    pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
    pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())
    protocol.delay(seconds=5)

 Input mass of vial after transfer (mf). Code will calculate the relative error of transfer

In [ ]:
mf = 37.9461

m = mf-mi
error = (m-density*volume/1000)/(density/1000*volume)*100
df.iloc[-1,2] = volume
df.iloc[-1, 0] = liquid
df.iloc[-1, 1] = pipette
df.iloc[-1,-6] = density
df.iloc[-1,-5] = mi
df.iloc[-1,-4] = mf
df.iloc[-1, -3] = m
df.iloc[-1,-2]= error

In [ ]:
#Update liquid level - if lower than 15, update
liquid_level = liquid_level - 2*volume/1000
print(liquid_level)

In [ ]:
#Observe error made
df

Repeat a total a ten times for each of  the following volumes 1000, 500 and 300 

In [ ]:
df.to_csv('liquid_'+str(liquid)+'_calibration_optimized.csv', index = False)

## Auxiliary code

In [ ]:
#drop tip in the bin
pipettes['p1000'].drop_tip()
pipettes['p1000'].home()

In [ ]:
#pick up a new tip
pipettes['p1000'].pick_up_tip()

In [ ]:
#remove residue from pipette tip
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].home_plunger()
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()
pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])

pipettes['p1000'].home_plunger()

pipettes['p1000'].blow_out(location = source.wells_by_name()['A1'].top())
pipettes['p1000'].touch_tip(location = source.wells_by_name()['A1'])
pipettes['p1000'].move_to(source.wells_by_name()['A1'].top())
